In [ ]:
#importing modules
from random import choices, randint, randrange, random
import os
from typing import List, Optional, Callable, Tuple
import numpy as np
from collections import namedtuple
from functools import partial
from Digits_StructureText_writer import StructureTextWriter
from Digits_freq_writer import Frequncies_writer
from Digits_MumaxFile_Writer import MumaxFileWriter
from Digit_Intensity_Calculator import IntensityCalculation #set path here
from Digit_Intensity_Calculator import calculate_intensities_all_arms

In [ ]:
#parameters
Genome = List[int]
Population = List[Genome]
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Genome], int]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]

In [ ]:
# generate required functions
def generate_genome(length: int) -> Genome:
    return choices([0, 1], weights=[85,15], k=length)


def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]

#selection of pair to be parent of next generation.
def selection_pair(population: Population, fitness_list) -> Population:
    return choices(
        population=population,
        weights=fitness_list,
        k=2     #draw twice per population
    )


#changing the gnomes
def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]


#mutation the population  
def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome


def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    return sum([fitness_func(genome) for genome in population])



def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
    return sorted(population, key=fitness_func, reverse=True)


def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))


def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), 
                           fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]

In [ ]:
def fitness(genome, generation_name, step_name):
    #print(genome)
    #Writing structure file
    struct_text = StructureTextWriter(struture_array=np.array(genome))
    struct_text.square_removel()
    struct_text.struture_text
    
    f = open('overview_G{}S{}.txt'.format(generation_name, step_name),'w')
    position_of_max = []
    
    #Writing Freuquncy file
    ftt_func = []
    for digits in range(3):
        if digits!=8:
            freq_writer = Frequncies_writer(digit=digits)
            freq_writer.write_frequncies()
            freq_text = freq_writer.frequncy_string + freq_writer.mask_string
            
            inputfile = 'G{}s{}f{}'.format(generation_name, step_name, digits)
            mumaxfile = MumaxFileWriter(
                filename='Digit_NN_allsides_512x128_OOP_0mT.mx3',
                structre_text= struct_text.struture_text,
                frequncy_text= freq_text,
                writefilename= inputfile+'.mx3'
                        )
            mumaxfile.file_writing()
            run_command = 'mumax3 -i {}'.format(inputfile+'.mx3')
            os.system(run_command)

            #calculating the intensities at all arms
            all_intensities = calculate_intensities_all_arms(inputfile+'.out/m000005.ovf')
            if max(all_intensities)/sum(all_intensities) > 0.5:
                ftt_func.append(1)
            else:
                ftt_func.append(max(all_intensities)*0.5)
            #all_intensities = list(np.random.sample(9))
            position_of_max.append(all_intensities.index(max(all_intensities)))
            f.write('digit{} '.format(digits)+ str(all_intensities) + '\n')  
    f.close()
    return len(set(position_of_max))

In [ ]:
#running the functions
def run_evolution(
        populate_func: PopulateFunc,
        fitness_func: FitnessFunc,
        fitness_limit: int,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    
    population = populate_func()
    
    for i in range(generation_limit):        
        fitnes = []
        for ind, gnom in enumerate(population):
            fitnes.append(fitness_func(gnom, i, ind))
            
        population = [x for _, x in sorted(zip(fitnes, population),reverse=True)]

        if printer is not None:
            printer(population, i, fitness_func)

        if max(fitnes) >= fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents     = selection_func(population, fitness_list=fitnes)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i

In [ ]:
population, generations = run_evolution(
    populate_func=partial(generate_population, size=5, genome_length=100),
    fitness_func =partial(fitness),
    fitness_limit=3,
    generation_limit=20
    )